In [ ]:
import datetime
import shutil
from pathlib import Path
from collections import Counter

import yaml
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.model_selection import KFold
import torch

In [ ]:

# model = YOLO('yolov9c.pt')

# # Display model information (optional)
# model.info()

# results = model.predict('TestSets\\Videos\\IMG_6946.mp4', save=True)

# # Check if results is a list and convert it to YOLO object



In [ ]:

if torch.cuda.is_available():
    torch.cuda.set_device(0)  # Set to your desired GPU number
else:
    print("No CUDA GPUs are available.")


In [ ]:
# label extraction
dataset_path = Path('datasets/yolo') 
labels = sorted((dataset_path / 'labels'/'train').rglob("*.txt")) 
print(labels)

In [ ]:

#label extraction
classes = ['Car','Cyclist','Pedestrian','Truck','Van','Tram']
cls_idx = [0,1,2,3,4,5]
indx = [l.stem for l in labels]
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)


In [ ]:
for label in labels:
    lbl_counter = Counter()

    with open(label,'r') as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(' ')[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`

print(labels_df)

In [ ]:
ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)   # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [ ]:
#creating folds
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
    folds_df[f'split_{idx}'].loc[labels_df.iloc[val].index] = 'val'

In [ ]:
# gathering fold distribution for later analysis
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

In [ ]:


# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files

images.extend(sorted((dataset_path / 'images'/'train').rglob(f"*png")))
print(images)

# Create the necessary directories and dataset YAML files (unchanged)
save_path = (dataset_path / f'K-Fold_Cross-val')
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'labels').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'labels').mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f'{split}_dataset.yaml'
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, 'w') as ds_y:
        Path = split_dir.as_posix()
        Path = Path.replace("datasets","")
        yaml.safe_dump({
            'path': Path,
            'train': 'train',
            'val': 'val',
            'names': classes
        }, ds_y)

In [ ]:
for image, label in zip(images, labels):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        print(split, k_split)
        img_to_path = save_path / split / k_split / 'images'
        lbl_to_path = save_path / split / k_split / 'labels'

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

In [13]:
import os
def get_newest_folder(directory):
    return max((os.path.join(directory, d) for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))), key=os.path.getctime)


weights_path = get_newest_folder(f'runs/detect') 
print(weights_path + '/weights/best.pt')

model = YOLO(weights_path + '/weights/best.pt')
model.val(data="val-config.yaml",project='val')

runs/detect\train46/weights/best.pt
Ultralytics YOLOv8.1.45  Python-3.11.3 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
Model summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\James\Documents\automtive-cv\automotive-cv\datasets\yolo\labels\test... 1988 images, 40 backgrounds, 0 corrupt: 100%|██████████| 1988/1988 [00:03<00:00, 512.33it/s]


val: New cache created: C:\Users\James\Documents\automtive-cv\automotive-cv\datasets\yolo\labels\test.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 125/125 [00:12<00:00,  9.92it/s]


                   all       1988      11477      0.888      0.611      0.675      0.495
                   Car       1988       6834      0.947       0.55      0.636      0.506
               Cyclist       1988        488      0.905      0.752       0.81      0.608
            Pedestrian       1988       2879        0.9      0.753      0.865      0.535
                 Truck       1988        297      0.809      0.182       0.21      0.155
                   Van       1988        829      0.915      0.556      0.631       0.48
                  Tram       1988        150      0.855      0.873      0.896      0.688
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to val\val2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001B24BF5F4D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
 

In [1]:

import os
import torch
from ultralytics import YOLO
import gc

torch.cuda.set_device(0)
weights_path = 'yolov8m.pt' 
# pretrained model can be changed to yolov8n.pt, for faster training

results={}

batch = 16
epochs = 10

#20 per fold
path = str(os.getcwd())

def get_newest_folder(directory):
    return max((os.path.join(directory, d) for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))), key=os.path.getctime)


for k in range(5):
# Your code here
    model = YOLO(weights_path)
    file_path = (f'{path}/datasets/yolo/K-Fold_Cross-val/split_{k+1}/split_{k+1}_dataset.yaml')    
    print(f'fold - {k+1}')
    model.train(mixup=0.1,data=file_path, batch=batch,epochs=epochs, device="gpu")  # include any train arguments

    del model
    gc.collect()
    torch.cuda.empty_cache()
    
    
    weights_path = get_newest_folder(f'runs/detect') + '/weights/last.pt'
    print(weights_path)
   
    


fold - 1
New https://pypi.org/project/ultralytics/8.1.46 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.45  Python-3.11.3 torch-2.2.2+cu121 CUDA:gpu (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=c:\Users\James\Documents\automtive-cv\automotive-cv/datasets/yolo/K-Fold_Cross-val/split_1/split_1_dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=gpu, workers=8, project=None, name=train49, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, vi

train: Scanning C:\Users\James\Documents\automtive-cv\automotive-cv\datasets\yolo\K-Fold_Cross-val\split_1\train\labels.cache... 2328 images, 2474 backgrounds, 0 corrupt: 100%|██████████| 4762/4762 [00:00<?, ?it/s]
val: Scanning C:\Users\James\Documents\automtive-cv\automotive-cv\datasets\yolo\K-Fold_Cross-val\split_1\val\labels.cache... 170 images, 1008 backgrounds, 0 corrupt: 100%|██████████| 1174/1174 [00:00<?, ?it/s]


Plotting labels to runs\detect\train49\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train49
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      6.49G      1.497      2.169      1.187          9        640: 100%|██████████| 298/298 [01:13<00:00,  4.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:04<00:00,  7.73it/s]

                   all       1174       1133      0.193      0.362      0.172     0.0903


...

       4/10      6.75G      1.091     0.9575      1.046          4        640: 100%|██████████| 299/299 [01:06<00:00,  4.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:04<00:00,  8.90it/s]

                   all       1166       1078      0.246       0.64      0.307      0.217



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/10      6.72G      1.056     0.9618      1.029          0        640: 100%|██████████| 299/299 [01:07<00:00,  4.44it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:04<00:00,  8.54it/s]

                   all       1166       1078      0.255      0.682      0.304      0.222
